# ⭐ Quality & Satisfaction Metrics Dashboard

**Purpose**: Analyze quality indicators, on-time delivery, and customer satisfaction metrics.

## Key Metrics
- On-Time Delivery percentage
- First-Time-Right rate
- Customer satisfaction scores
- Rejection/Objection analysis

---

## 1. Setup & Data Loading

In [ ]:
# Install required packages
!pip install pandas openpyxl plotly seaborn matplotlib wordcloud -q

import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pd.set_option('display.max_columns', None)
print("✅ Libraries loaded successfully!")

In [ ]:
# File path - update this to match your file location
filename = r"C:\Users\bmalaraju\Documents\WP-OP Agent\JIRA-Agent\11.25.WP Orders_25-11-2025_v01.xlsx"
print(f"📁 Using file: {filename}")

In [ ]:
# Load data
df = pd.read_excel(filename, engine='openpyxl')
print(f"📊 Dataset loaded: {len(df):,} rows, {len(df.columns)} columns")
print(f"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

## 2. Data Preparation

In [ ]:
# Column mapping for quality metrics
COLUMN_MAP = {
    'in_time_delivery': 'In-Time Delivery',
    'satisfaction': 'Survey Satisfaction Mark',
    'first_right': 'Survey First Right',
    'suggestion': 'Survey Suggestion',
    'rejection_reason': 'Approved/Rejected Reason',
    'status': 'WP Order Status',
    'product': 'Product',
    'customer': 'Customer',
    'order_id': 'WP Order ID',
    'quantity': 'WP Quantity',
    'completed_qty': 'WP Completed Qty'
}

# Check which columns exist
available = {k: v for k, v in COLUMN_MAP.items() if v in df.columns}
missing = {k: v for k, v in COLUMN_MAP.items() if v not in df.columns}

print("✅ Available quality columns:")
for k, v in available.items():
    print(f"   - {v}")

if missing:
    print(f"\n⚠️ Missing columns (will be skipped):")
    for k, v in missing.items():
        print(f"   - {v}")

In [ ]:
# Prepare analysis dataframe
analysis_df = df.copy()
total_orders = len(analysis_df)

# Normalize In-Time Delivery
itd_col = COLUMN_MAP.get('in_time_delivery')
if itd_col and itd_col in analysis_df.columns:
    analysis_df['in_time_clean'] = analysis_df[itd_col].fillna('').astype(str).str.strip().str.lower()
    analysis_df['is_on_time'] = analysis_df['in_time_clean'].isin(['yes', 'y', 'true', '1'])
    analysis_df['is_late'] = analysis_df['in_time_clean'].isin(['no', 'n', 'false', '0'])
    analysis_df['itd_known'] = analysis_df['is_on_time'] | analysis_df['is_late']
else:
    analysis_df['itd_known'] = False

# Normalize First-Time-Right
ftr_col = COLUMN_MAP.get('first_right')
if ftr_col and ftr_col in analysis_df.columns:
    analysis_df['ftr_clean'] = analysis_df[ftr_col].fillna('').astype(str).str.strip().str.lower()
    analysis_df['is_first_right'] = analysis_df['ftr_clean'].isin(['yes', 'y', 'true', '1'])
    analysis_df['ftr_known'] = analysis_df['ftr_clean'].isin(['yes', 'y', 'true', '1', 'no', 'n', 'false', '0'])
else:
    analysis_df['ftr_known'] = False

# Parse satisfaction score
sat_col = COLUMN_MAP.get('satisfaction')
if sat_col and sat_col in analysis_df.columns:
    analysis_df['satisfaction_score'] = pd.to_numeric(analysis_df[sat_col], errors='coerce')
    analysis_df['sat_known'] = analysis_df['satisfaction_score'].notna()
else:
    analysis_df['sat_known'] = False

print("\n✅ Quality data prepared!")

## 3. Quality KPIs Overview

In [ ]:
# Calculate quality metrics
# On-Time Delivery
itd_total = analysis_df['itd_known'].sum()
on_time = analysis_df['is_on_time'].sum() if 'is_on_time' in analysis_df.columns else 0
on_time_pct = (on_time / itd_total * 100) if itd_total > 0 else 0

# First-Time-Right
ftr_total = analysis_df['ftr_known'].sum()
first_right = analysis_df['is_first_right'].sum() if 'is_first_right' in analysis_df.columns else 0
ftr_pct = (first_right / ftr_total * 100) if ftr_total > 0 else 0

# Satisfaction
sat_total = analysis_df['sat_known'].sum()
avg_satisfaction = analysis_df.loc[analysis_df['sat_known'], 'satisfaction_score'].mean() if sat_total > 0 else 0

# Display summary
print("="*60)
print("⭐ QUALITY METRICS SUMMARY")
print("="*60)
print(f"\n📦 Total Orders: {total_orders:,}")
print(f"\n⏱️ ON-TIME DELIVERY:")
print(f"   Data Available:   {itd_total:,} orders ({itd_total/total_orders*100:.1f}%)")
print(f"   On-Time Rate:     {on_time_pct:.1f}%")
print(f"\n✅ FIRST-TIME-RIGHT:")
print(f"   Data Available:   {ftr_total:,} orders ({ftr_total/total_orders*100:.1f}%)")
print(f"   FTR Rate:         {ftr_pct:.1f}%")
print(f"\n⭐ CUSTOMER SATISFACTION:")
print(f"   Surveys:          {sat_total:,} orders ({sat_total/total_orders*100:.1f}%)")
print(f"   Avg Score:        {avg_satisfaction:.2f}")
print("="*60)

In [ ]:
# Quality KPI Gauges
fig = make_subplots(
    rows=1, cols=3,
    specs=[[{'type': 'indicator'}, {'type': 'indicator'}, {'type': 'indicator'}]],
    subplot_titles=['On-Time Delivery', 'First-Time-Right', 'Avg Satisfaction']
)

# On-Time Delivery Gauge
fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=on_time_pct,
    gauge={
        'axis': {'range': [0, 100]},
        'bar': {'color': '#4ECDC4'},
        'steps': [
            {'range': [0, 60], 'color': '#FFCDD2'},
            {'range': [60, 80], 'color': '#FFF9C4'},
            {'range': [80, 100], 'color': '#C8E6C9'}
        ],
        'threshold': {'line': {'color': 'green', 'width': 4}, 'thickness': 0.75, 'value': 90}
    },
    number={'suffix': '%'}
), row=1, col=1)

# First-Time-Right Gauge
fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=ftr_pct,
    gauge={
        'axis': {'range': [0, 100]},
        'bar': {'color': '#2ECC71'},
        'steps': [
            {'range': [0, 60], 'color': '#FFCDD2'},
            {'range': [60, 80], 'color': '#FFF9C4'},
            {'range': [80, 100], 'color': '#C8E6C9'}
        ],
        'threshold': {'line': {'color': 'green', 'width': 4}, 'thickness': 0.75, 'value': 90}
    },
    number={'suffix': '%'}
), row=1, col=2)

# Satisfaction Gauge (assuming 1-5 or 1-10 scale)
max_score = 5 if avg_satisfaction <= 5 else 10
fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=avg_satisfaction,
    gauge={
        'axis': {'range': [0, max_score]},
        'bar': {'color': '#9B59B6'},
        'steps': [
            {'range': [0, max_score*0.4], 'color': '#FFCDD2'},
            {'range': [max_score*0.4, max_score*0.7], 'color': '#FFF9C4'},
            {'range': [max_score*0.7, max_score], 'color': '#C8E6C9'}
        ]
    },
    number={'valueformat': '.2f'}
), row=1, col=3)

fig.update_layout(height=350, title={'text': 'Quality Performance Indicators', 'x': 0.5})
fig.show()

## 4. On-Time Delivery Analysis

In [ ]:
# On-Time Delivery pie chart
if itd_total > 0:
    late_count = analysis_df['is_late'].sum() if 'is_late' in analysis_df.columns else 0
    unknown_count = total_orders - itd_total
    
    labels = ['✅ On-Time', '❌ Late', '❓ Unknown']
    values = [on_time, late_count, unknown_count]
    colors = ['#4ECDC4', '#FF6B6B', '#95A5A6']
    
    fig = go.Figure(data=[go.Pie(
        labels=labels,
        values=values,
        hole=0.4,
        marker_colors=colors,
        textinfo='percent+value'
    )])
    
    fig.update_layout(
        title={'text': 'On-Time Delivery Distribution', 'x': 0.5, 'font': {'size': 20}},
        annotations=[{'text': f'{on_time_pct:.0f}%<br>On-Time', 'x': 0.5, 'y': 0.5, 'font_size': 16, 'showarrow': False}],
        height=450
    )
    
    fig.show()
else:
    print("⚠️ No On-Time Delivery data available")

In [ ]:
# On-Time Delivery by Product
product_col = COLUMN_MAP.get('product')

if itd_total > 0 and product_col and product_col in analysis_df.columns:
    itd_by_product = analysis_df[analysis_df['itd_known']].groupby(product_col).agg(
        total=('is_on_time', 'count'),
        on_time=('is_on_time', 'sum')
    ).reset_index()
    
    itd_by_product['on_time_pct'] = (itd_by_product['on_time'] / itd_by_product['total'] * 100).round(1)
    itd_by_product = itd_by_product[itd_by_product['total'] >= 5]  # Minimum 5 orders
    itd_by_product = itd_by_product.sort_values('on_time_pct', ascending=True).tail(15)
    
    fig = go.Figure(data=[go.Bar(
        y=itd_by_product[product_col],
        x=itd_by_product['on_time_pct'],
        orientation='h',
        marker_color=np.where(itd_by_product['on_time_pct'] >= 80, '#4ECDC4', 
                              np.where(itd_by_product['on_time_pct'] >= 60, '#FFE66D', '#FF6B6B')),
        text=itd_by_product['on_time_pct'].apply(lambda x: f'{x:.0f}%'),
        textposition='outside'
    )])
    
    fig.update_layout(
        title={'text': 'On-Time Delivery Rate by Product', 'x': 0.5, 'font': {'size': 20}},
        xaxis_title='On-Time %',
        yaxis_title='Product',
        height=500,
        margin={'l': 200}
    )
    
    # Add target line
    fig.add_vline(x=90, line_dash="dash", line_color="green", annotation_text="Target: 90%")
    
    fig.show()

## 5. Customer Satisfaction Analysis

In [ ]:
# Satisfaction score distribution
if sat_total > 0:
    sat_data = analysis_df[analysis_df['sat_known']]['satisfaction_score']
    
    fig = go.Figure(data=[go.Histogram(
        x=sat_data,
        nbinsx=10,
        marker_color='#9B59B6'
    )])
    
    fig.update_layout(
        title={'text': 'Satisfaction Score Distribution', 'x': 0.5, 'font': {'size': 20}},
        xaxis_title='Satisfaction Score',
        yaxis_title='Number of Responses',
        height=400
    )
    
    # Add mean line
    fig.add_vline(x=avg_satisfaction, line_dash="dash", line_color="red",
                  annotation_text=f"Mean: {avg_satisfaction:.2f}")
    
    fig.show()
    
    # Statistics
    print(f"\n📊 Satisfaction Statistics:")
    print(f"   Mean:   {sat_data.mean():.2f}")
    print(f"   Median: {sat_data.median():.2f}")
    print(f"   Std:    {sat_data.std():.2f}")
    print(f"   Min:    {sat_data.min():.2f}")
    print(f"   Max:    {sat_data.max():.2f}")
else:
    print("⚠️ No satisfaction score data available")

In [ ]:
# Satisfaction by Product
if sat_total > 0 and product_col and product_col in analysis_df.columns:
    sat_by_product = analysis_df[analysis_df['sat_known']].groupby(product_col).agg(
        responses=('satisfaction_score', 'count'),
        avg_score=('satisfaction_score', 'mean'),
        min_score=('satisfaction_score', 'min'),
        max_score=('satisfaction_score', 'max')
    ).reset_index()
    
    sat_by_product = sat_by_product[sat_by_product['responses'] >= 3]  # Minimum 3 responses
    sat_by_product = sat_by_product.sort_values('avg_score', ascending=True).tail(15)
    
    fig = go.Figure()
    
    # Box representing min-max range
    for _, row in sat_by_product.iterrows():
        color = '#4ECDC4' if row['avg_score'] >= 4 else ('#FFE66D' if row['avg_score'] >= 3 else '#FF6B6B')
        fig.add_trace(go.Bar(
            y=[row[product_col]],
            x=[row['avg_score']],
            orientation='h',
            marker_color=color,
            text=f"{row['avg_score']:.2f}",
            textposition='outside',
            showlegend=False
        ))
    
    fig.update_layout(
        title={'text': 'Average Satisfaction by Product', 'x': 0.5, 'font': {'size': 20}},
        xaxis_title='Average Satisfaction Score',
        yaxis_title='Product',
        height=500,
        margin={'l': 200},
        barmode='overlay'
    )
    
    fig.show()

## 6. Rejection & Objection Analysis

In [ ]:
# Rejection reasons analysis
status_col = COLUMN_MAP.get('status')
reason_col = COLUMN_MAP.get('rejection_reason')

if status_col and status_col in analysis_df.columns:
    rejected_df = analysis_df[analysis_df[status_col].isin(['Rejected', 'Objected'])]
    
    print(f"\n📊 REJECTION/OBJECTION SUMMARY")
    print(f"   Rejected Orders:  {(analysis_df[status_col] == 'Rejected').sum():,}")
    print(f"   Objected Orders:  {(analysis_df[status_col] == 'Objected').sum():,}")
    print(f"   Total Issues:     {len(rejected_df):,}")
    
    if reason_col and reason_col in analysis_df.columns and len(rejected_df) > 0:
        # Filter to orders with reasons
        with_reasons = rejected_df[
            rejected_df[reason_col].notna() & 
            (rejected_df[reason_col].str.strip() != '')
        ]
        
        if len(with_reasons) > 0:
            reason_counts = with_reasons[reason_col].value_counts().head(10)
            
            fig = go.Figure(data=[go.Bar(
                x=reason_counts.values,
                y=reason_counts.index,
                orientation='h',
                marker_color='#E74C3C',
                text=reason_counts.values,
                textposition='outside'
            )])
            
            fig.update_layout(
                title={'text': 'Top Rejection/Objection Reasons', 'x': 0.5, 'font': {'size': 20}},
                xaxis_title='Count',
                yaxis_title='Reason',
                height=400,
                margin={'l': 300},
                yaxis={'categoryorder': 'total ascending'}
            )
            
            fig.show()
        else:
            print("\n⚠️ No rejection reasons recorded")

In [ ]:
# Rejection rate trend (if date available)
if 'Added Date' in analysis_df.columns and status_col and status_col in analysis_df.columns:
    trend_df = analysis_df.copy()
    trend_df['added_date'] = pd.to_datetime(trend_df['Added Date'], errors='coerce')
    trend_df = trend_df[trend_df['added_date'].notna()]
    trend_df['week'] = trend_df['added_date'].dt.to_period('W').dt.start_time
    
    weekly_quality = trend_df.groupby('week').agg(
        total=('WP Order ID', 'count'),
        rejected=(status_col, lambda x: (x == 'Rejected').sum()),
        objected=(status_col, lambda x: (x == 'Objected').sum())
    ).reset_index()
    
    weekly_quality['issue_rate'] = ((weekly_quality['rejected'] + weekly_quality['objected']) / weekly_quality['total'] * 100).round(1)
    weekly_quality = weekly_quality.tail(12)
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=weekly_quality['week'],
        y=weekly_quality['issue_rate'],
        mode='lines+markers',
        name='Issue Rate %',
        line=dict(color='#E74C3C', width=2)
    ))
    
    fig.update_layout(
        title={'text': 'Weekly Rejection/Objection Rate Trend', 'x': 0.5, 'font': {'size': 20}},
        xaxis_title='Week',
        yaxis_title='Issue Rate %',
        height=400
    )
    
    fig.show()

## 7. Quality Correlation Analysis

In [ ]:
# Correlation: On-Time Delivery vs Satisfaction
if itd_total > 0 and sat_total > 0:
    corr_df = analysis_df[analysis_df['itd_known'] & analysis_df['sat_known']].copy()
    
    if len(corr_df) > 0:
        # Group by on-time status
        correlation = corr_df.groupby('is_on_time').agg(
            count=('satisfaction_score', 'count'),
            avg_satisfaction=('satisfaction_score', 'mean')
        ).reset_index()
        correlation['Status'] = correlation['is_on_time'].map({True: 'On-Time', False: 'Late'})
        
        fig = go.Figure(data=[go.Bar(
            x=correlation['Status'],
            y=correlation['avg_satisfaction'],
            marker_color=['#4ECDC4', '#FF6B6B'],
            text=correlation['avg_satisfaction'].round(2),
            textposition='outside'
        )])
        
        fig.update_layout(
            title={'text': 'Satisfaction Score by Delivery Status', 'x': 0.5, 'font': {'size': 20}},
            xaxis_title='Delivery Status',
            yaxis_title='Average Satisfaction Score',
            height=400
        )
        
        fig.show()
        
        print(f"\n📊 Correlation Insight:")
        on_time_sat = corr_df[corr_df['is_on_time']]['satisfaction_score'].mean()
        late_sat = corr_df[~corr_df['is_on_time']]['satisfaction_score'].mean()
        diff = on_time_sat - late_sat
        print(f"   On-Time orders have {abs(diff):.2f} {'higher' if diff > 0 else 'lower'} satisfaction on average")
else:
    print("⚠️ Insufficient data for correlation analysis")

## 8. Survey Feedback Analysis

In [ ]:
# Survey suggestions word cloud
suggestion_col = COLUMN_MAP.get('suggestion')

if suggestion_col and suggestion_col in analysis_df.columns:
    suggestions = analysis_df[
        analysis_df[suggestion_col].notna() & 
        (analysis_df[suggestion_col].str.strip() != '')
    ][suggestion_col].tolist()
    
    if len(suggestions) > 0:
        from wordcloud import WordCloud
        import matplotlib.pyplot as plt
        
        text = ' '.join(str(s) for s in suggestions)
        
        wordcloud = WordCloud(
            width=800, height=400,
            background_color='white',
            colormap='viridis',
            max_words=100
        ).generate(text)
        
        plt.figure(figsize=(12, 6))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.title('Customer Suggestion Themes', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        print(f"\n📝 Total Suggestions Received: {len(suggestions):,}")
    else:
        print("⚠️ No survey suggestions recorded")
else:
    print("⚠️ Survey suggestion column not found")

## 9. Quality Summary Table

In [ ]:
# Quality metrics by product
if product_col and product_col in analysis_df.columns:
    quality_summary = analysis_df.groupby(product_col).agg(
        total_orders=('itd_known', 'count'),
        itd_known=('itd_known', 'sum'),
        on_time=('is_on_time', 'sum') if 'is_on_time' in analysis_df.columns else (product_col, lambda x: 0),
        sat_count=('sat_known', 'sum'),
        avg_sat=('satisfaction_score', 'mean')
    ).reset_index()
    
    quality_summary['on_time_pct'] = np.where(
        quality_summary['itd_known'] > 0,
        (quality_summary['on_time'] / quality_summary['itd_known'] * 100).round(1),
        np.nan
    )
    
    quality_summary = quality_summary.sort_values('total_orders', ascending=False)
    
    print("\n📊 QUALITY METRICS BY PRODUCT (Top 15)")
    print("="*100)
    display_df = quality_summary.head(15)[[
        product_col, 'total_orders', 'on_time_pct', 'sat_count', 'avg_sat'
    ]].copy()
    display_df.columns = ['Product', 'Total Orders', 'On-Time %', 'Surveys', 'Avg Satisfaction']
    display_df['Avg Satisfaction'] = display_df['Avg Satisfaction'].round(2)
    print(display_df.to_string(index=False))

## 10. Export Results

In [ ]:
# Export to Excel
export_filename = f"quality_satisfaction_{datetime.now().strftime('%Y%m%d_%H%M')}.xlsx"

with pd.ExcelWriter(export_filename, engine='openpyxl') as writer:
    # Summary
    summary_data = pd.DataFrame({
        'Metric': ['Total Orders', 'On-Time Delivery Data', 'On-Time Rate',
                   'First-Time-Right Data', 'FTR Rate',
                   'Satisfaction Surveys', 'Avg Satisfaction'],
        'Value': [total_orders, itd_total, f'{on_time_pct:.1f}%',
                  ftr_total, f'{ftr_pct:.1f}%',
                  sat_total, f'{avg_satisfaction:.2f}']
    })
    summary_data.to_excel(writer, sheet_name='Summary', index=False)
    
    # By Product
    if 'quality_summary' in dir():
        quality_summary.to_excel(writer, sheet_name='By Product', index=False)

print(f"\n✅ Results exported to: {export_filename}")
# files.download() - uncomment if using Colab
# files.download(export_filename)

---

## 📋 Summary

This notebook analyzed quality and customer satisfaction metrics:

| Metric | Description |
|--------|-------------|
| **On-Time Delivery** | % of orders delivered by requested date |
| **First-Time-Right** | % of orders completed correctly first time |
| **Satisfaction Score** | Average customer survey rating |
| **Rejection Rate** | % of orders rejected |
| **Objection Rate** | % of orders with customer objections |

### Key Insights
1. Track on-time delivery improvement over time
2. Address products with low satisfaction scores
3. Investigate common rejection reasons
4. Correlate quality metrics with customer satisfaction